In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# In[ ]:



# In[2]:


from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import T5Config
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from transformers import set_seed


# In[3]:


from datetime import datetime
import argparse
import os
import sys

sys.path.append("./hf_transformers/")


# In[4]:

import torch

from data_reader import GetDataAsPython
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time


# In[34]:


import torch

from data_reader import GetDataAsPython, MinimalDataPoint
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time
import csv

start_all = datetime.now()

# In[6]:

import socket
local = False if 'computecanada' in socket.gethostname() else True

base_model = 'training/t5-small_repo-based_21-01-2022_10-29-42/checkpoint-16440'

if local:
    storage_directory = './storage/'
    load_model = f'./{storage_directory}/{base_model}'
    batch_size = 16
else:
    storage_directory = '/scratch/arminz/'
    batch_size = 64
    load_model = f'/{storage_directory}/{base_model}'

# In[7]:

In [23]:

import random


# In[8]:


exec_number = random.randint(0, 1000)
exec_number


# In[31]:


# parser = argparse.ArgumentParser()
# parser.add_argument("-r", "--repo", type=str, default='/data/all/data/oroinc/platform')
# parser.add_argument("-p", "--percent", type=float, default=1)

# args = parser.parse_args()
repo = '/data/all/data/emberjs/ember.js'#args.repo
#sample_percent = args.percent

print('start:', repo)

lr = 4e-3
ws = 300
wd = 0.4
print('best arguments', lr, wd, ws)




# In[35]:


name='good'
name



start: /data/all/data/emberjs/ember.js
best arguments 0.004 0.4 300


'good'

In [24]:
# Read and prepare data
test_data = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_repo_specific_final.json")
test_data_eslint = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_eslint_final.json")
test_data += test_data_eslint

In [25]:
len(test_data)


# In[38]:


all_warning_types = extract_warning_types(test_data)


# In[39]:


(_train_inputs, _train_labels, _val_inputs, _val_labels, test_inputs, test_labels, train_info, val_info, test_info, ) = \
    create_data(test_data, all_warning_types, include_warning=True, design='repo-based-included', select_repo=repo)

splitting by : repo-based-included
train size: 129
val size: 42
test size: 47


In [26]:
train_data = MinimalDataPoint.FromJsonToPython(f'{storage_directory}/bt_data/{repo}.json')

In [27]:
(train_inputs, train_labels, val_inputs, val_labels, _test_inputs, _test_labels, train_info, val_info, test_info, ) = \
    create_data(train_data, all_warning_types, include_warning=True, design='repo-based-included', select_repo=repo)


splitting by : repo-based-included
train size: 27
val size: 10
test size: 10


In [28]:

tokenizer = T5Tokenizer.from_pretrained(load_model)


# In[41]:




In [29]:

# Create dataset required by pytorch
# samples = int(sample_percent * len(train_inputs))
train_dataset = create_dataset(train_inputs[:], train_labels[:], tokenizer, pad_truncate=True, max_length=128)
val_dataset = create_dataset(val_inputs, val_labels, tokenizer, pad_truncate=True)

# print(f'amount of data that is being used for fine-tuning (train) : {len(train_dataset)} == {samples} ({sample_percent})')
print(f'amount of data that is being used for fine-tuning (validation): {len(val_dataset)} (full)')
print(f'amount of data that will be probably being used for testing: {sum([len(x) for x in test_inputs.values()])} (full)')

amount of data that is being used for fine-tuning (validation): 10 (full)
amount of data that will be probably being used for testing: 47 (full)


In [30]:
now = datetime.now()
full_name = f'{name}_{exec_number}_{repo.rsplit("/", 1)[1][-20:]}'
model_directory = f'{storage_directory}/tmp/bt-finetuned/{full_name}'
model_directory


'./storage//tmp/bt-finetuned/good_114_ember.js'

In [31]:
tokenizer = T5Tokenizer.from_pretrained(load_model)
model = T5ForConditionalGeneration.from_pretrained(load_model)
model.resize_token_embeddings(len(tokenizer))
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32104, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32104, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [32]:

training_args = Seq2SeqTrainingArguments(
    output_dir=model_directory,
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=ws,
    weight_decay=wd,
    logging_dir=model_directory,
    logging_steps=100,
    do_eval=True,
    evaluation_strategy="epoch",
    learning_rate=lr,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
    eval_accumulation_steps=1,  # set this lower, if testing or validation crashes
    disable_tqdm=False,
    predict_with_generate=True,  # never set this to false.
    seed=42,  # default value
)

In [33]:

# In[68]:
from transformers import EarlyStoppingCallback


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=[torch.optim.Adam(params=model.parameters(), lr=lr), None],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)



In [34]:
trainer.train()


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.119863,0.023700,421.415000
2,No log,0.119360,0.023500,425.325000
3,No log,0.119164,0.024100,414.707000
4,No log,0.119586,0.023400,426.610000
5,No log,0.120144,0.023600,423.252000
6,No log,0.120627,0.023900,418.159000
7,No log,0.122918,0.024100,415.274000


TrainOutput(global_step=14, training_loss=0.07772886753082275, metrics={'train_runtime': 9.4901, 'train_samples_per_second': 6.322, 'total_flos': 8712273282048, 'epoch': 7.0})

In [35]:
trainer.evaluate()

{'eval_loss': 0.11916355788707733,
 'eval_runtime': 0.0332,
 'eval_samples_per_second': 301.155,
 'epoch': 7.0}

In [36]:
model_directory

'./storage//tmp/bt-finetuned/good_114_ember.js'

In [37]:
tuned_model_dir=f'{storage_directory}/tmp/finetuned/' + repo
trainer.save_model(tuned_model_dir)

In [38]:
os.system(f'python hf_transformers/tfix_testing.py --load-model {tuned_model_dir} -bs 16 --model-name t5-small -d repo-based-included -r {repo}')


/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
12it [00:07,  2.00it/s]              

start time:  18:50:57
['no-invalid-this', 'no-throw-literal', 'no-new-wrappers', 'guard-for-in', 'no-new-object', 'comma-style', 'prefer-spread', 'no-caller', 'no-extra-bind', 'no-array-constructor', 'prefer-rest-params', 'generator-star-spacing', 'no-this-before-super', 'no-extend-native', 'no-undef', 'no-useless-escape', 'no-dupe-keys', 'no-console', 'no-constant-condition', 'no-duplicate-case', 'no-empty', 'no-extra-semi', 'no-redeclare', 'no-cond-assign', 'no-extra-boolean-cast', 'no-fallthrough', 'no-unreachable', 'valid-typeof', 'no-unsafe-finally', 'no-unused-vars', 'no-debugger', 'no-unsafe-negation', 'no-case-declarations', 'no-self-assign', 'no-process-exit', 'no-inner-declarations', 'for-direction', 'no-compare-neg-zero', 'no-sparse-arrays', 'no-func-assign', 'no-const-assign', 'no-global-assign', 'use-isnan', 'no-unused-labels', 'require-yield', 'getter-return', 'no-dupe-class-members', 'no-ex-assign', 'constructor-super', 'no-new-symbol', 'no-empty-pattern', 'no-class-assi

0